# Generador de Datos AY06

Pueden usar este generador para generar el archivo `dummy_data.json`.

Es posible que no tengan alunas libreras necesarias, pero pueden crear un `venv` e instalarlas (`pip install -r requirements.txt`)

In [1]:
from faker import Faker
import numpy as np
from json import dump
from random import shuffle
from datetime import datetime
from dateutil.relativedelta import relativedelta


fake = Faker()

N = 100

In [2]:
# https://stackoverflow.com/a/16025584/16384621

def generate_samples(n=1000, cov=[[0.3, 0.2], [0.2, 0.2]], num_variables = 2):
    L = np.linalg.cholesky(cov)

    uncorrelated = np.random.standard_normal((num_variables, n))
    mean = [1, 1]
    correlated = np.dot(L, uncorrelated) + np.array(mean).reshape(2, 1)

    return np.concatenate([correlated, uncorrelated]).T

In [4]:
data = []

for i, (var1, var2, var3, var4) in enumerate(generate_samples(n=N)):
    end_date = datetime.now()
    start_date = end_date - relativedelta(years=1)
    random_date = fake.date_time_between(start_date="-1y", end_date="now")
    dif = (end_date.year - random_date.year) * 12 + end_date.month - random_date.month
    exp_dif = pow(2, dif)
    exp_dif_with_error = np.random.normal(loc = exp_dif, scale = 0.5 * exp_dif)

    datum = {
        "id": i,
        "var_1": var1,
        "var_2": var2,
        "var_3": var3,
        "var_4": var4,
        "var_5": max(0.1, exp_dif_with_error),
        "var_6": random_date.isoformat()
    }
    data.append(datum)

meta_metadata = {
        "var_1": "Variable #1",
        "var_2": "Variable #2",
        "var_3": "Variable #3",
        "var_4": "Variable #4",
        "var_5": "Variable #5",
        "var_6": "Variable #6",
        "id": "Id"
    }

names = [{"key": k, "name": v} for k, v in meta_metadata.items()]
scaleTypes = ["Linear", "Exponential", "Date"]
shuffle(data)

with open("dummy_data.json", "w", encoding="utf-8") as file:
    dump({
        "data": data,
        "metadata": {
            "names": names,
            "scaleTypes": scaleTypes},
    }, file)
